In [1]:
state = 'IA' 
year = 2020
obj_type = 'perimeter'
starting_deviation = 0.01 

In [2]:
import sys, os
src_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(src_path)

In [3]:
filepath = '../../dat/' + str(year) + '/'
filename = state + '_county.json'
filename2 = state + '_county.shp'

In [4]:
from read import read_graph_from_json

G = read_graph_from_json(state, filepath + filename, year=year)
print(f"The state of {state} has {G._k} districts.")
G._ideal_population = sum(G.nodes[i]['TOTPOP'] for i in G.nodes) / G._k

The state of IA has 4 districts.


In [5]:
#import warm starts
sys.path.append(os.path.abspath('../heuristic'))

from IA_plans_2020 import plans
print(f"Loaded {len(plans)} plans from file.")
warm_starts = plans

Loaded 18 plans from file.


In [6]:
from pareto import filter_and_sort_pareto
from metrics import scores

plans_scores = [scores(G, plan, G._ideal_population, obj_type) for plan in warm_starts]
_,_,nondominated_warm_starts_plans = filter_and_sort_pareto(plans=warm_starts, upper_bounds=plans_scores, obj_type=obj_type)
print(f"Selected {len(nondominated_warm_starts_plans)} nondominated warm start plans")

Selected 13 nondominated warm start plans


In [ ]:
from epsilon_constraint import epsilon_constraint_method

(plans, obj_bounds, deviations) = epsilon_constraint_method(
            G,                 
            obj_type,          
            contiguity = 'lcut',                                             # {'lcut', 'scf', 'shir'} 
            cutoff=None,       
            verbose= True,
            warm_start_mode = 'user',                                        # {'None', 'user', 'refinement'}
            warm_starts=warm_starts,                                         # if you have user define warm starts else it is None
            starting_deviation=starting_deviation, 
            time_limit=7200, 
            sizes=None,      
            max_B=True,                                                      # If symmetry_breaking is 'orbitope' or you have warm_start, max_B should be True   
            symmetry_breaking='orbitope',                                    # {None, 'orbitope', 'rsum'} 
            state=state,
            year=year
        )

Initially, L = 789617 and U = 805568 and k = 4.

****************************************
Trying deviation = 7975.922500000001
****************************************
Using user-provided warm starts.
Selected warm_start = [[1, 2, 8, 14, 20, 21, 22, 27, 41, 59, 60, 65, 68, 71, 76, 78, 79, 84, 92], [3, 10, 12, 18, 24, 25, 26, 28, 38, 39, 44, 46, 47, 48, 54, 61, 67, 83, 86, 94, 95, 96, 97, 98], [0, 4, 9, 16, 17, 19, 23, 30, 31, 32, 33, 35, 36, 37, 40, 42, 43, 49, 50, 52, 53, 55, 57, 58, 62, 63, 64, 66, 70, 73, 74, 81, 89, 91, 93], [5, 6, 7, 11, 13, 15, 29, 34, 45, 51, 56, 69, 72, 75, 77, 80, 82, 85, 87, 88, 90]]
Objective value: 38.90543801945935
Deviation: 663.25

****************************************
Running labeling model!
****************************************
L = 789617 and U = 805568
Set parameter Username
Set parameter LicenseID to value 2608266
Academic license - for non-commercial use only - expires 2026-01-09
sizes =  [1, 1, 1, 1]
Solving the max B problem (as MIP) for use

Solving the max B problem (as MIP) for use in the vertex ordering...
Set parameter LogToConsole to value 0
Set parameter LazyConstraints to value 1
Applying warm start!
Set parameter MIPGap to value 0
Set parameter FeasibilityTol to value 1e-07
Set parameter TimeLimit to value 7200
Set parameter IntFeasTol to value 1e-07
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  7200
FeasibilityTol  1e-07
IntFeasTol  1e-07
MIPGap  0
LazyConstraints  1

Optimize a model with 4751 rows, 4200 columns and 16291 nonzeros
Model fingerprint: 0xef74f7b6
Variable types: 1188 continuous, 3012 integer (3012 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+05]
  Objective range  [2e-02, 7e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+05]

Us

 21068  3682     cutoff   31        37.59760   34.64254  7.86%   190  344s
 21801  3699   35.85680   32  544   37.59760   34.70045  7.71%   190  351s
 22553  3712   37.03594   30  514   37.59760   34.75779  7.55%   190  355s
 23443  3708   37.26998   30  370   37.59760   34.80613  7.42%   189  363s
 24402  3687   37.43849   29  496   37.59760   34.88822  7.21%   189  370s
 25324  3678   37.19962   33  433   37.59760   34.94853  7.05%   189  376s
 26343  3618     cutoff   28        37.59760   35.03593  6.81%   188  387s
 27311  3570   36.95831   29  367   37.59760   35.11943  6.59%   188  397s
 28381  3501 infeasible   28        37.59760   35.20217  6.37%   188  406s
 29410  3439   36.17830   28  513   37.59760   35.30437  6.10%   188  418s
 30564  3314   37.40123   38  435   37.59760   35.38401  5.89%   188  428s
 31741  3180 infeasible   32        37.59760   35.47321  5.65%   188  440s
 32978  2994   37.34439   37  390   37.59760   35.56636  5.40%   188  451s
 34147  2802     cutoff  

  7230  2285   37.89485   28  516   38.42881   31.62192  17.7%   201  202s
  7444  2307   35.95217   21  514   38.42881   31.62192  17.7%   200  206s
  7901  2523   38.37876   28  532   38.42881   31.62192  17.7%   199  212s
  8211  2674   37.47493   29  395   38.42881   31.64063  17.7%   197  217s
  8552  2796   37.67580   27  444   38.42881   31.65955  17.6%   195  220s
  9182  3075   35.17442   22  478   38.42881   32.06483  16.6%   195  227s
  9515  3138 infeasible   28        38.42881   32.12797  16.4%   194  231s
  9871  3218   36.86758   25  518   38.42881   32.12797  16.4%   193  236s
 10296  3308   34.29072   23  493   38.42881   32.20002  16.2%   192  241s
 10684  3391   36.59295   20  506   38.42881   32.38653  15.7%   191  247s
 11056  3519   35.97314   23  468   38.42881   32.40799  15.7%   190  252s
 11476  3593   38.11698   33  298   38.42881   32.53761  15.3%   190  257s
 11892  3723   36.07734   24  531   38.42881   32.61798  15.1%   190  262s
 12318  3826   36.88022  

Coefficient statistics:
  Matrix range     [1e+00, 5e+05]
  Objective range  [2e-02, 7e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+05]

User MIP start produced solution with objective 38.9054 (0.20s)
Loaded user MIP start with objective 38.9054

Presolve removed 2223 rows and 1798 columns
Presolve time: 0.39s
Presolved: 2528 rows, 2402 columns, 9179 nonzeros
Variable types: 0 continuous, 2402 integer (2375 binary)

Root relaxation: objective 2.553941e+01, 1634 iterations, 0.98 seconds (0.09 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   25.53941    0  488   38.90544   25.53941  34.4%     -    2s
     0     0   25.67604    0  500   38.90544   25.67604  34.0%     -    3s
     0     0   25.88005    0  511   38.90544   25.88005  33.5%     -    5s
     0     0   25.88056    0  511   38.90544   25.88056  33.5%     -    7s
     0     0   

...

 684006 238656     cutoff   63        42.43317   38.97472  8.15%   147 5198s
 686794 239743   41.50342   32  253   42.43317   38.97867  8.14%   147 5221s
 690045 240388   41.95008   48   98   42.43317   38.98170  8.13%   147 5237s
 692924 241174   40.24782   31  342   42.43317   38.98543  8.13%   147 5256s
 695365 242174   41.06073   39  271   42.43317   38.98774  8.12%   147 5269s
 698245 242556   40.60191   47  166   42.43317   38.99183  8.11%   146 5285s
 699444 243616   40.20443   50  208   42.43317   38.99225  8.11%   146 5304s
 701042 243616     cutoff   32        42.43317   38.99225  8.11%   146 5305s
 702430 244479   41.43220   49  184   42.43317   38.99620  8.10%   146 5317s
 705370 245425     cutoff   54        42.43317   38.99936  8.09%   146 5336s
 708257 246483   41.36408   37  387   42.43317   39.00414  8.08%   146 5351s
 711294 247548   41.14719   48  137   42.43317   39.00764  8.07%   146 5370s
 714447 248305   41.48467   29  462   42.43317   39.01104  8.06%   146 5386s

 972744 327925     cutoff   41        42.43317   39.26410  7.47%   137 7010s
 975558 328934   41.05604   43  364   42.43317   39.26641  7.46%   137 7032s
 978614 329759     cutoff   33        42.43317   39.26940  7.46%   137 7050s
 981369 330715   40.78460   46  148   42.43317   39.27133  7.45%   137 7071s
 984414 331638   41.93778   29  420   42.43317   39.27384  7.45%   137 7089s
 987354 332667 infeasible   49        42.43317   39.27575  7.44%   137 7109s
 990352 333613   41.73385   29  404   42.43317   39.27902  7.43%   137 7127s
 993458 334569 infeasible   72        42.43317   39.28197  7.43%   137 7147s
 996515 335169     cutoff   38        42.43317   39.28439  7.42%   137 7165s
 998845 336033   41.83853   61  111   42.43317   39.28570  7.42%   137 7180s
 1001672 336753   41.13842   43   99   42.43317   39.28792  7.41%   137 7200s

Cutting planes:
  Lazy constraints: 4

Explored 1004060 nodes (137101905 simplex iterations) in 7203.54 seconds (917.10 work units)
Thread count was 20

 12045  6255   36.45958   25  552   42.46268   32.93598  22.4%   161  214s
 12476  6429     cutoff   34        42.46268   32.93598  22.4%   162  217s
 13255  6925   37.60626   27  465   42.46268   33.18608  21.8%   163  224s
 13742  7152 infeasible   41        42.46268   33.18608  21.8%   164  231s
 14247  7486   42.19930   34  645   42.46268   33.31251  21.5%   163  238s
 14871  7769   37.12457   24  502   42.46268   33.47190  21.2%   163  245s
 15465  7806   36.46619   26  531   42.46268   33.57442  20.9%   163  253s
 15548  8020   38.25464   29  440   42.46268   33.57442  20.9%   163  259s
 15946  8221   37.94628   28  493   42.46268   33.57442  20.9%   162  266s
 16730  8521   39.00791   30  428   42.46268   33.62135  20.8%   161  272s
 17489  8776   38.03275   26  612   42.46268   33.71940  20.6%   161  280s
 17935  9005   36.85668   28  555   42.46268   33.75944  20.5%   162  290s
 18753  9283   41.54081   56   75   42.46268   33.75944  20.5%   161  295s
 19393  9645     cutoff  

 218247 84546   40.18709   32  267   42.46268   37.78661  11.0%   151 2124s
 220878 85339   38.58760   70  203   42.46268   37.80600  11.0%   150 2144s
 223362 86078   42.32191   32  144   42.46268   37.81482  10.9%   150 2163s
 225761 87057   41.59786   34  459   42.46268   37.82647  10.9%   150 2186s
 228380 87989   41.41948   27  436   42.46268   37.84452  10.9%   150 2208s
 231032 89101   39.65022   34  260   42.46268   37.85867  10.8%   150 2226s
 233882 90063   41.12183   36  267   42.46268   37.87029  10.8%   149 2245s
 236501 91316   41.19636   55   73   42.46268   37.88467  10.8%   149 2265s
 239601 92068   40.48397   31  469   42.46268   37.90077  10.7%   149 2286s
 242108 93051   40.87704   36  137   42.46268   37.91278  10.7%   148 2305s
 244770 93937   42.13738   30  176   42.46268   37.92305  10.7%   148 2321s
 247297 94878   40.44490   41  331   42.46268   37.93311  10.7%   148 2343s
 249895 95727 infeasible   38        42.46268   37.94965  10.6%   148 2361s
 252390 9657

 500059 186361   41.50296   31  251   42.46268   38.65411  8.97%   132 4140s
 501880 187418   41.50529   57  218   42.46268   38.65824  8.96%   132 4163s
 504862 188116 infeasible   35        42.46268   38.66393  8.95%   132 4178s
 507600 189033   42.04842   46  312   42.46268   38.66669  8.94%   132 4201s
 510406 189833   42.03196   42  213   42.46268   38.67296  8.92%   131 4216s
 512943 190857   40.56633   29  593   42.46268   38.67843  8.91%   131 4239s
 515668 191819     cutoff   49        42.46268   38.68377  8.90%   131 4256s
 518476 192776   40.54045   57  156   42.46268   38.68829  8.89%   131 4275s
 521174 193421   40.57439   33  291   42.46268   38.69502  8.87%   131 4293s
 523673 194302   41.55638   37  151   42.46268   38.69878  8.86%   131 4311s
 526272 195434   41.41185   54  269   42.46268   38.70439  8.85%   131 4331s
 529154 196221     cutoff   55        42.46268   38.71073  8.84%   131 4350s
 531875 196639   41.46053   56  228   42.46268   38.71402  8.83%   131 4368s

In [ ]:
result = list(zip(plans, obj_bounds , deviations))

In [ ]:
epsilon = 1 / (2 * G._k)
min_deviation = min(round(r[2], 1) for r in result)

if min_deviation < epsilon:
    no_solution_region = None
else:
    no_solution_region = [0, min_deviation]

if no_solution_region is not None:
    print(f"No feasible solution was found within the region: {no_solution_region}")

In [ ]:
from pareto import plot_pareto_frontiers

plot_pareto_frontiers(
                G,
                method='epsilon_constraint_method',
                plans=None,                                   #if method ='epsilon_constraint_method' is None 
                obj_types=obj_type,                               
                ideal_population=G._ideal_population,
                state=state,
                filepath=filepath,
                filename2=filename2,
                no_solution_region=no_solution_region,
                year=year,
                result=result                               #if method ='heuristic' is None 
             )

In [ ]:
from draw import draw_plan
from metrics import observed_deviation_persons, compute_obj

print(f"\n{'#' * 100}\nPareto maps for state {state}, objective {obj_type}\n{'#' * 100}\n")

format_obj = {
    'bottleneck_Polsby_Popper': lambda x: round(1 /x, 4),
    'cut_edges': lambda x: int(x)}
G._L = 0
G._U = G._k * G._ideal_population

for plan, obj_bound, dev in result:
    obs_dev = observed_deviation_persons(G, plan, G._ideal_population)
    obj = compute_obj(G, plan, obj_type)
    obj_val = format_obj.get(obj_type, lambda x: round(x, 4))(obj)
    deviation_percentage = round(100 * dev / G._ideal_population, 4)
    title = f"{round(obs_dev, 2)}-person deviation ({deviation_percentage}%), with {obj_val} {obj_type}"
    draw_plan(filepath, filename2, G, plan, title=title, year=year)